In [1]:
import pprint
from datetime import datetime
import pandas as pd
import os
import rasterio as rs
from rasterio.features import shapes
import geopandas as gpd
import numpy as np
from scipy.ndimage import label, sum as ndi_sum, minimum_filter, generate_binary_structure
import subprocess
import yaml
from osgeo import gdal
from osgeo import osr
from osgeo import ogr
from osgeo import gdalconst
import sys
from scipy import ndimage
sys.path.append('../src/')
# import prepare_rasters as pr
import prepare_polys as pp
import prepare_rasters as pr
import prepare_polys_orig as preprocess

%load_ext autoreload
%autoreload 2

## Prototyping
What are the input requirements?
1. inputParams
2. TTC - these look the same
3. lulc 2016 - slightly differently sizes bc of method for resampling/reprj. no use of compression

In [18]:
## quick comp of inputs (after preprocessing)
j_lulc = rs.open('../data/costa_rica/interim/lulc_cr_reprj.tif').read(1)
j_ttc = rs.open('../data/costa_rica/raw/CostaRica.tif').read(1)
j_sdpt = rs.open('../data/costa_rica/interim/cri_sdpt_v2.tif').read(1)
j_lulc.shape, j_ttc.shape, j_sdpt.shape

((35683, 37670), (33372, 37119), (33372, 37119))

In [3]:
preprocess.preprocess_datasets('../data/Input/lulc.tif')

Creating output file that is 37119P x 33372L.
Processing ../data/Input/lulc.tif [1/1] : 0Using internal nodata values (e.g. -1) for image ../data/Input/lulc.tif.
Copying nodata values from source ../data/Input/lulc.tif to destination ../data/Input/lulc_reprj.tif.
...10...20...30...40...50...60...70...80...90...100 - done.


In [26]:
e_lulc = rs.open('../data/Input/lulc_reprj.tif').read(1)
e_ttc = rs.open('../data/Input/TTC.tif').read(1)
e_lulc.shape, e_ttc.shape

((33372, 37119), (33372, 37119))

In [28]:
j_lulc.shape[0] - e_lulc.shape[0]

2311

In [29]:
j_lulc.shape[1] - e_lulc.shape[1]

551

In [9]:
pp.create_highconf_polygons(params_path='../data/Input/inputParams.csv',
                            lulc_path = '../data/Input/lulc_reprj.tif',
                            sdpt_path = '../data/costa_rica/interim/cri_sdpt_v2.tif', 
                            ttc_path = '../data/Input/TTC.tif'
                           )

Merging all high conf polygons...
All polygons have been merged and saved to ../data/Output/Final/merged_polygons.shp
Statistics saved to ../data/Output/Final/stats_jessica4.csv


In [6]:
af_poly = gpd.read_file('../data/Output/Intermediate/Poly/Agro_Coffee.shp')
af_poly

,raster_val,geometry
0,1.0,"POLYGON ((-84.33506 10.15646, -84.33497 10.156..."
1,1.0,"POLYGON ((-84.33497 10.15637, -84.33488 10.156..."
2,1.0,"POLYGON ((-84.33506 10.15619, -84.33497 10.156..."
3,1.0,"POLYGON ((-84.33305 10.15512, -84.33296 10.155..."
4,1.0,"POLYGON ((-84.33159 10.15314, -84.33150 10.153..."
...,...,...
240,1.0,"POLYGON ((-82.89391 8.77387, -82.89382 8.77387..."
241,1.0,"POLYGON ((-82.89492 8.77333, -82.89482 8.77333..."
242,1.0,"POLYGON ((-82.89437 8.77459, -82.89419 8.77459..."
243,1.0,"POLYGON ((-82.89501 8.77324, -82.89492 8.77324..."


In [ ]:
# get the starting shape of the AF raster
# and the count of polygons
af_raster = rs.open('../data/Output/Intermediate/Rasters/Agro_Coffee.tif').read(1)
af_poly = gpd.read_file('../data/Output/Intermediate/Poly/Agro_Coffee.shp')
af_raster.shape, af_poly.shape

In [34]:
# we know af raster is comprised of 1s and 0s
np.unique(af_raster, return_counts=True)

(array([0, 1], dtype=uint8), array([1238688044,      47224]))

In [41]:
# now label and get count for each label
labeled_array, num_features = ndimage.label(af_raster)

In [42]:
# get pixel count for each label
label_size = [(labeled_array == label).sum() for label in range(num_features + 1)]


KeyboardInterrupt



In [ ]:
label_size

In [ ]:
thresh = 196

In [ ]:
for label,size in enumerate(label_size):
    if size < thresh:
        arr[labeled_array == label] = 0

In [ ]:
    filtered_image = image * mask

    results = (
        {'properties': {'raster_val': v}, 'geometry': s}
        for i, (s, v) in enumerate(shapes(filtered_image,
                                        mask=filtered_image, 
                                        transform=src.transform))
                                        )

    geoms = list(results)
    if geoms:
        gdf = gpd.GeoDataFrame.from_features(geoms)
        gdf.crs = crs  # set the CRS for the GeoDataFrame
        gdf.to_file(os.path.join(output_folder, filename.replace('.tif', '.shp')))

# connected component analysis

In [4]:
def label_components(category,
                     min_size=196,
                     input_folder =  '../data/Output/Intermediate/Rasters/',
                     output_folder = '../data/Output/Intermediate/Poly/'):
    """
    Convert raster files to polygons, only including areas with a 
    minimum number of connected pixels.
    Diagonal connections are not considered.

    :param min_size: Minimum number of connected pixels to be included
    :return: None
    """
    with rs.open(input_folder + f"{category}.tif") as src:
        image = src.read(1)  
        crs = src.crs  
        out_meta = src.meta.copy() 
        out_meta.update({'compress':'lzw'})

       # Label connected components
        structure = generate_binary_structure(2, 2)
        labeled, _ = label(image, structure=structure) # label connected components in the raster.
        sizes = ndi_sum(image, labeled, range(labeled.max() + 1)) #calculate the size of the components
        mask = np.isin(labeled, np.where(sizes > min_size)[0]) # create a mask to filter out components less than min size

        filtered_image = image * mask

    # save min filtered for comparison
    with rs.open(input_folder + f"{category}_cca_test2.tif",'w',**out_meta, BIGTIFF='YES') as dst:
        dst.write(filtered_image, indexes=1)
        
        # results = (
        #     {'properties': {'raster_val': v}, 'geometry': s}
        #     for i, (s, v) in enumerate(shapes(filtered_image,
        #                                     mask=filtered_image, 
        #                                     transform=src.transform))
        #                                     )
        # geoms = list(results)
        # gdf = gpd.GeoDataFrame.from_features(geoms)
        # gdf.crs = crs  # set the CRS for the GeoDataFrame
        # gdf.to_file(os.path.join(output_folder + f"{category}.shp"))
    
    return None

In [7]:
struc = generate_binary_structure(2, 2)

In [8]:
struc

array([[ True,  True,  True],
       [ True,  True,  True],
       [ True,  True,  True]])

In [5]:
label_components(category="Mono_Palm")

In [3]:
def min_filter(category,
                     input_folder =  '../data/Output/Intermediate/Rasters/',
                     output_folder = '../data/Output/Intermediate/Poly/'):
    
    """
    applies a minimum filter of 14x14 to the raster
    in order to identify eligible sized polygons
    """
    with rs.open(input_folder + f"{category}.tif") as src:
        image = src.read(1) 
        crs = src.crs 
        out_meta = src.meta.copy() 
        out_meta.update({'compress':'lzw'})

        # will return 1 if area fits given size otherwise 0
        min_filtered = minimum_filter(image, (14,14))
        
        print(f"starting count: {np.unique(image, return_counts=True)}")
        print(f"post filter count: {np.unique(min_filtered, return_counts=True)}")

    # save min filtered for comparison
    with rs.open(input_folder + f"{category}_cca_test.tif",'w',**out_meta, BIGTIFF='YES') as dst:
        dst.write(min_filtered, indexes=1)
        
        # Generate polygons from the filtered raster
        # shapes converts the filtered raster (min_filtered) 
        # into a sequence of geometry-value pairs.
        # results = (
        #     {'properties': {'raster_val': v}, 'geometry': s}
        #     for i, (s, v) in enumerate(shapes(arr, transform=src.transform)))
        
        # Create a GeoDataFrame from the features
        # gdf = gpd.GeoDataFrame.from_features(results, crs=crs)
        # gdf.to_file(os.path.join(output_folder + f"{category}.shp"))
        
    return None

In [4]:
arr = rastertopoly_new(category="Mono_Palm")

starting count: (array([0, 1], dtype=uint8), array([1236217817,    2517451]))
post filter count: (array([0, 1], dtype=uint8), array([1237766529,     968739]))


In [15]:
arr

,geometry,raster_val
0,"POLYGON ((-83.59910 10.54266, -83.59910 10.542...",1.0
1,"POLYGON ((-83.60696 10.54095, -83.60696 10.540...",1.0
2,"POLYGON ((-83.59783 10.54095, -83.59783 10.540...",1.0
3,"POLYGON ((-83.60193 10.53924, -83.60193 10.539...",1.0
4,"POLYGON ((-83.59856 10.54023, -83.59856 10.540...",1.0
...,...,...
1065,"POLYGON ((-82.96083 8.40934, -82.96083 8.40925...",1.0
1066,"POLYGON ((-82.96795 8.41087, -82.96795 8.41078...",1.0
1067,"POLYGON ((-82.97389 8.40557, -82.97389 8.40548...",1.0
1068,"POLYGON ((-82.96978 8.40224, -82.96978 8.40170...",1.0


In [17]:
arr.to_file(os.path.join("../data/Output/Intermediate/Poly/Mono_Palm_CCA.shp"))

In [10]:
arr.shape

(33372, 37119)

In [11]:
np.unique(arr, return_counts=True)

(array([0, 1], dtype=uint8), array([1237766529,     968739]))

# Extract centroids

In [2]:
hc1 = gpd.read_file('../data/costa_rica/interim/v1/hc_poly_v1.shp')
hc2 = gpd.read_file('../data/costa_rica/interim/v2/hc_poly_v2.shp')

In [3]:
# to compare 
hc_pts1 = gpd.read_file('../data/costa_rica/interim/v1/labeled_pts.shp')
print(hc1.shape, hc2.shape)

(2194, 9) (151828, 4)


In [8]:
hc1.PClass.value_counts()

PClass
notplantation    1789
monoculture       317
agroforestry       88
Name: count, dtype: int64

In [5]:
hc2.label.value_counts()

label
3    131878
0     17017
1      2688
2       245
Name: count, dtype: int64